In [1]:
import pandas as pd

## Ingestion

In [2]:
data = pd.read_csv('./data/cleaned_data.csv', sep=';')

In [3]:
data.head()

,id,type_d_article,numéro_de_l_article_ou_de_la_loi,description_ou_texte_complet,mots_clés_ou_sujets_abordés,date_de_publication,source
0,0,Loi,Loi du 23 octobre 2018,La loi entend renforcer la lutte contre la fra...,"fraude fiscale, fraude sociale, fraude douanière",25 octobre 2018,Légifrance
1,1,Loi,Loi du 23 octobre 2018,La loi a été promulguée le 23 octobre 2018. El...,"fraude fiscale, fraude sociale, fraude douanière",24 octobre 2018,Journal officiel
2,2,Loi,Loi du 23 octobre 2018,La loi complète la loi du 10 août 2018 pour un...,"fraude fiscale, fraude sociale, fraude douanière",23 octobre 2018,Légifrance
3,3,Loi,Loi du 23 octobre 2018,"Elle prévoit la création par décret d'une ""pol...","fraude fiscale, fraude sociale, fraude douanière",23 octobre 2018,Légifrance
4,4,Loi,Loi du 23 octobre 2018,Le texte renforce les moyens de détection et d...,"fraude fiscale, fraude sociale, fraude douanière",23 octobre 2018,Légifrance


In [4]:
data.columns

Index(['id', 'type_d_article', 'numéro_de_l_article_ou_de_la_loi',
       'description_ou_texte_complet', 'mots_clés_ou_sujets_abordés',
       'date_de_publication', 'source'],
      dtype='object')

In [5]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-11-20 16:15:56--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.74K  --.-KB/s    in 0.002s  

2024-11-20 16:15:56 (2.14 MB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [6]:
data.columns

Index(['id', 'type_d_article', 'numéro_de_l_article_ou_de_la_loi',
       'description_ou_texte_complet', 'mots_clés_ou_sujets_abordés',
       'date_de_publication', 'source'],
      dtype='object')

In [7]:
import minsearch
index = minsearch.Index(
    text_fields=['type_d_article', 'numéro_de_l_article_ou_de_la_loi',
       'description_ou_texte_complet', 'mots_clés_ou_sujets_abordés',
       'date_de_publication', 'source'],
    keyword_fields=[]
)

In [8]:
documents = data.to_dict(orient="records")

In [9]:
index.fit(documents)

In [10]:
query = "Quels sont les textes des lois sur les fraudes fiscales?"

In [11]:
index.search(query, num_results=2)

[{'id': 17,
  'type_d_article': 'Loi',
  'numéro_de_l_article_ou_de_la_loi': 'Loi du 6 décembre 2013',
  'description_ou_texte_complet': 'Les articles 1729 et 1741 du CGI régissent respectivement les sanctions fiscales et pénales pour des faits de fraude fiscale.',
  'mots_clés_ou_sujets_abordés': 'fraude fiscale',
  'date_de_publication': '6 décembre 2013',
  'source': "Ministère de l'Economie, des Finances et de la Relance"},
 {'id': 20,
  'type_d_article': 'Loi',
  'numéro_de_l_article_ou_de_la_loi': 'Loi du 6 décembre 2013',
  'description_ou_texte_complet': "Encadrement du cumul. Les conditions du cumul de ces sanctions pénales et fiscales ont été encadrées par le Conseil constitutionnel qui a estimé qu'il devait être réservé aux cas les plus graves de dissimulation des sommes soumises à l'impôts. La gravité se démontre par le montant des droits éludés, la nature des agissements du contribuable ou les circonstances dans lesquelles la fraude a eu lieu.",
  'mots_clés_ou_sujets_abor

## RAG Flow

In [ ]:
import os 

os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

In [13]:
from openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{"role": "user", "content": query}]
)

response.choices[0].message.content

"Les lois sur les fraudes fiscales varient selon les pays, mais en général, elles incluent des dispositions dans le Code des impôts et des lois pénales. Dans le cas de la France, par exemple, plusieurs textes régissent la lutte contre la fraude fiscale :\n\n1. **Code général des impôts (CGI)** : Ce code contient des dispositions relatives à la détermination des bases d’imposition, des obligations déclaratives et des sanctions en cas de fraude.\n\n2. **Code pénal** : Les articles concernant les délits économiques et financiers, en particulier les articles relatifs à la fraude fiscale, stipulent les sanctions pénales pour les comportements frauduleux.\n\n3. **Loi n° 2013-1117 du 6 décembre 2013** : Cette loi renforce les mesures de lutte contre la fraude fiscale, notamment en matière de sanctions.\n\n4. **Loi de finances** : Chaque année, des lois de finances peuvent introduire des mesures spécifiques sur la fraude fiscale.\n\n5. **Directive de l'Union européenne** : Certaines directives

In [14]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [15]:
prompt_template = """
You are an expert assistant in French tax law. Answer the QUESTION using only the information provided in the CONTEXT, 
which is sourced from official legal texts and databases about tax fraud in France. Provide accurate, factual answers that comply with French law. 
If the QUESTION goes beyond the information available in the CONTEXT, state that you cannot answer without additional information.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
type_d_article : {type_d_article}
numéro_de_l_article_ou_de_la_loi : {numéro_de_l_article_ou_de_la_loi}
description_ou_texte_complet : {description_ou_texte_complet}  
mots_clés_ou_sujets_abordés : {mots_clés_ou_sujets_abordés}
date_de_publication : {date_de_publication}
source : {source}
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [16]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [17]:
def rag(query, model='gpt-4o-mini'):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    return answer

In [18]:
question = "Quelles sont les sanctions prévues pour une fraude fiscale impliquant un compte bancaire à l'étranger non déclaré ?"
answer = rag(question)
print(answer)

Les sanctions prévues pour une fraude fiscale impliquant un compte bancaire à l'étranger non déclaré sont les suivantes :

1. **Peine d'emprisonnement** : L'auteur du délit de fraude fiscale peut encourir jusqu'à 5 ans d'emprisonnement, selon la loi du 14 mars 2012.
   
2. **Amende** : Une amende de 500 000 euros peut également être imposée pour ce délit.

3. **Circonstances aggravantes** : Si la fraude est commise en bande organisée, les sanctions peuvent être portées à 7 ans d'emprisonnement et une amende de 30 000 000 euros, selon la loi du 6 décembre 2013.

Ces sanctions sont applicables dans le cadre des articles 1729 et 1741 du code général des impôts (CGI), qui régissent les sanctions fiscales et pénales.


In [19]:
# une question qui n'est pas dans la base de connaissance du modele
question = "Quand est-ce que ce poste de data scientist/data engineer à ursaaf expire ?"
answer = rag(question)
print(answer)

Je ne peux pas répondre à cette question sans informations supplémentaires.


In [20]:
# Questions 
question = "COMMENT DETECTER UNE FRAUDE FISCALE ?"
answer = rag(question)
print(answer)

La détection d'une fraude fiscale peut être facilitée par plusieurs dispositifs établis par la législation française, notamment :

1. **Dénonciation obligatoire** : La loi du 23 octobre 2018 a instauré un dispositif de dénonciation obligatoire des faits de fraude fiscale, ce qui aide à identifier plus efficacement les cas de fraude.

2. **Collaboration entre l'administration fiscale et le procureur** : Depuis 2018, un dialogue accru est encouragé entre l'administration fiscale et le procureur de la République, permettant ainsi un meilleur suivi des affaires de fraude fiscale, même au-delà des plaintes formelles.

3. **Poursuites et comparution sur reconnaissance préalable de culpabilité** : La loi de 2018 a étendu le recours à la comparution sur reconnaissance préalable de culpabilité pour les délits de fraude fiscale, permettant un traitement plus rapide des cas.

En outre, les sanctions aggravées en cas de fraude commise en bande organisée soulignent la gravité de certains comporteme

## Retrieval Evaluation

In [21]:
df_question = pd.read_csv('./data/ground-truth-retrieval.csv')
df_question.head()

,id,question
0,0,Quelle est l'objectif principal de la loi du 2...
1,0,Quand la loi du 23 octobre 2018 a-t-elle été p...
2,0,Quels types de fraude sont ciblés par la loi d...
3,0,D'où provient l'information relative à la loi ...
4,0,Quel est le type de texte légal de la loi du 2...


In [22]:
ground_truth = df_question.to_dict(orient='records')
ground_truth[0]

{'id': 0,
 'question': "Quelle est l'objectif principal de la loi du 23 octobre 2018 ?"}

In [23]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [24]:
def minsearch_search(query):
    boost = {}
    try:
        results = index.search(
            query=query,
            filter_dict={},
            boost_dict=boost,
            num_results=10
        )
    except Exception as e:
        print(f"Error searching for {query}: {e}")
        return []

    return results


In [25]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)

        # # Afficher le résultat de la recherche pour chaque question
        # print(f"Query: {q['question']}")
        # print(f"Results: {results}")

        # if results:
        #     print(f"First result structure: {results[0]}")

        # # Si aucun résultat n'est trouvé, vérifier si la recherche fonctionne correctement
        # if not results:
        #     print(f"No results for query: {q['question']}")

        # Comparer les ids correctement
        relevance = [d.get('id') == doc_id for d in results]
        # print(f"Relevance: {relevance}")
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }


In [26]:
from tqdm.auto import tqdm

In [27]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

  0%|          | 0/25 [00:00<?, ?it/s]

{'hit_rate': 0.48, 'mrr': 0.287047619047619}

### Finding the best parameters

In [28]:
df_validation = df_question[:10]
df_test = df_question[10:]

In [29]:
import random

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Assuming we're minimizing. Use float('-inf') if maximizing.

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

In [30]:
gt_val = df_validation.to_dict(orient='records')

In [31]:
def minsearch_search(query, boost=None):
    if boost is None:
        boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [32]:
param_ranges = {
    'type_d_article': (0.0, 3.0),
    'numéro_de_l_article_ou_de_la_loi': (0.0, 3.0),
    'description_ou_texte_complet': (0.0, 3.0),
    'mots_clés_ou_sujets_abordés': (0.0, 3.0),
    'date_de_publication': (0.0, 3.0),
    'source': (0.0, 3.0) 
}


def objective(boost_params):
    def search_function(q):
        return minsearch_search(q['question'], boost_params)

    results = evaluate(gt_val, search_function)
    return results['mrr']

In [33]:
simple_optimize(param_ranges, objective, n_iterations=20)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

({'type_d_article': 0.5938517881880572,
  'numéro_de_l_article_ou_de_la_loi': 0.3625600276274884,
  'description_ou_texte_complet': 1.3004076240628533,
  'mots_clés_ou_sujets_abordés': 2.0447509328918176,
  'date_de_publication': 2.753271735509114,
  'source': 2.105413779654186},
 0.19083333333333333)

In [34]:
def minsearch_improved(query):
    boost = {
        'type_d_article': 0.22,
        'numéro_de_l_article_ou_de_la_loi': 0.499,
        'description_ou_texte_complet': 1.97,
        'mots_clés_ou_sujets_abordés': 2.60,
        'date_de_publication': 0.80,
        'source': 0.42
    }

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )
    return results

evaluate(ground_truth, lambda q: minsearch_improved(q['question']))

  0%|          | 0/25 [00:00<?, ?it/s]

{'hit_rate': 0.56, 'mrr': 0.32433333333333336}

## RAG evaluation

In [35]:
prompt2_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [36]:
len(ground_truth)

25

In [37]:
ground_truth[0]

{'id': 0,
 'question': "Quelle est l'objectif principal de la loi du 23 octobre 2018 ?"}

In [38]:
record = ground_truth[0]

In [48]:
print(answer_llm)

La loi du 23 octobre 2018 a été publiée le 23 octobre 2018.


In [49]:
prompt = prompt2_template.format(question=question, answer_llm=answer_llm)
print(prompt)

You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: Quand la loi du 23 octobre 2018 a-t-elle été publiée ?
Generated Answer: La loi du 23 octobre 2018 a été publiée le 23 octobre 2018.

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}


In [42]:
import json
df_sample = df_question.sample(n=15, random_state=1)

In [43]:
sample = df_sample.to_dict(orient='records')

In [44]:
evaluations = []

for record in tqdm(sample):
    question = record['question']
    answer_llm = rag(question) 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)

    evaluations.append((record, answer_llm, evaluation))

  0%|          | 0/15 [00:00<?, ?it/s]

In [45]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

In [46]:
df_eval.relevance.value_counts(normalize=True)

relevance
RELEVANT    1.0
Name: proportion, dtype: float64

In [47]:
df_eval.to_csv('./data/rag-eval-gpt-4o-mini.csv', index=False)